In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math
try:
  import plotly.express as px
except:
  %pip install plotly
  import plotly.express as px


##Importing the file

In [56]:
big_df = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/big_df.csv').iloc[:,1:].drop(columns = ['Geo Point'])
#Make FIPS uniform with 5 numbers
big_df['POOFips'] = big_df['POOFips'].apply(lambda x:'0'+str(int(x)) if int(x)<10000 else str(int(x)))

In [6]:
big_df.head()

,X,Y,DiscoveryAcres,FireCause,POOFips,POOState,StartMonth,StartYear,Temp,Rainfall,Max_Temp,Total_Month,ALAND,AWATER,Geo_X,Geo_Y,POP_2019,POP_2020,POP_2021,POP_2022
0,-87.348925,34.101946,0.1,Human,01133,US-AL,10,2022,58.5,3.33,72.6,0.0,1587.614057,48.917203,34.15,-87.37,23689.0,23698.0,23491.0,23652.0
1,-87.367355,34.314726,1.0,Human,01079,US-AL,1,2021,42.2,2.89,51.7,0.0,1788.864251,68.682382,34.52,-87.31,33089.0,33055.0,33063.0,33090.0
2,-85.402504,32.654173,179.0,Unknown,01081,US-AL,11,2022,55.0,6.49,67.0,NaN,1573.514768,21.526461,32.60,-85.36,170423.0,172030.0,174601.0,177218.0
3,-85.892422,33.395450,0.1,Human,01027,US-AL,10,2019,66.0,5.96,77.2,0.0,1564.251834,5.285207,33.27,-85.86,14184.0,14256.0,14193.0,14190.0
4,-87.184715,32.823056,5.0,Human,01105,US-AL,10,2022,60.8,3.42,75.1,1.0,1863.900621,10.937769,32.64,-87.29,8856.0,8727.0,8452.0,8355.0


##A function to find out the counties where wildfires are frequent

In [7]:
def freq_wfire(big_df,state,yr1 = 2019, yr2=2022):
  '''
  Input: dataframe, state, year range.
  Output: list of counties where wildfires are very frequent, for the given state, over the years
  '''
  #Store all counties
  county_list = big_df[big_df['POOState']==state]['POOFips'].unique().tolist()
  #for each year, find intersection with the counties with high wfire frequency
  for yr in range(yr1,yr2+1):
    c_list = big_df[(big_df['POOState']==state) &(big_df['StartYear']==yr)  ]['POOFips'].value_counts().nlargest(15).index.tolist()
    county_list = set(county_list).intersection(c_list)
  #return the list of counties
  return list(county_list)

## Define an integer problem


```
Minimize sum_i sum_j d_{ij} x_{ij}
subject to:
  sum_j x_{ij} = 1 for all i 
  sum_i x_{ij} < k y_j
  sum_j y_j <= max_stations
  x_{ij}, y_j are binary for all i,j
```

> x_{ij} is 1 if location i looks over station j

> y_j itakes the value 1 if a station is built at location j and 0 otherwise

> d_{ij} is the distance between location i and station j

> Each station has a capacity k






## Using GUROBI to solve the integer program

In [ ]:
#Download packages if not present
try:
  import gurobipy as gp
  from gurobipy import GRB
except:
  %pip install gurobipy
  import gurobipy as gp
  from gurobipy import GRB

def opt_station_local(county_data):
  locations_df = county_data[['Y','X']]
  locations = {loc_id: (x, y) for loc_id, x, y in zip(locations_df.index.tolist(), locations_df['X'], locations_df['Y'])}
  location_to_idx = {i: idx for idx, i in enumerate(locations)}
  idx_to_location = {idx: i for i, idx in location_to_idx.items()}

  n = len(location_to_idx) 
  m = 11 # number of potential station locations
  k = 16 # max number of locations each station can serve
  max_stations = int(np.log(county_data['ALAND'].iloc[0])) +3 # max number of stations to build in a county

  stations = [(i, j) for i in range(m) for j in range(m)]

  d = {(idx,j): math.sqrt((locations[i][0]-stations[j][0])**2 + (locations[i][1]-stations[j][1])**2) for i,idx in location_to_idx.items() for j in range(m)}
  #In order to extend the project to include cost based on location, one can also define a cost vector
  #Update the objective function accordingly and we will get a complex model 
  #This could be made more dynamic as well if required.
  #c = {j: 10 for j in stations}

  # Create model
  #Need to use licensed version for bigger models
  model = gp.Model('minimum cost flow')
  model.setParam('OutputFlag',0)
  # Create variables
  x = model.addVars(n, m, vtype=GRB.BINARY, name='x')
  y = model.addVars(m, vtype=GRB.BINARY, name='y')

  # Set objective function
  obj = gp.quicksum(d[i,j] * x[i,j] for i in range(n) for j in range(m))
  model.setObjective(obj, GRB.MINIMIZE)

  # Add constraints
  model.addConstrs(gp.quicksum(x[i,j] for j in range(m)) == 1 for i in range(n)) # each location is served by one station
  model.addConstrs(gp.quicksum(x[i,j] for i in range(n)) <= k * y[j] for j in range(m)) # each station serves at most k locations
  model.addConstr(gp.quicksum(y[j] for j in range(m)) <= max_stations) # at most max_stations stations are built

  # Optimize model
  model.optimize()

  # Print solution
  if model.status == GRB.OPTIMAL:
      station_coords = []
      for j in range(m):
          if y[j].x > 0.5:
              locations_served = [i for i in range(n) if x[i,j].x > 0.5]
              station_coords.append(stations[j])
  else:
      print('No solution found.')
      return
  station_coord = []
  station_idx = [i for i,x in enumerate(stations) if x in station_coords]
  for j in station_idx:
    assigned_loc = [i for i in range(n) if np.abs(x[i,j].X-1).round(1)==0]
    if len(assigned_loc)==0:
        continue
    else:
        dist = 100
        station_lat = sum(locations_df.loc[idx_to_location[i]][0] for i in assigned_loc)/len(assigned_loc)
        station_lon = sum(locations_df.loc[idx_to_location[i]][1] for i in assigned_loc)/len(assigned_loc)
        for p in station_coord:
          val = np.abs(p[0] -station_lat) +np.abs(p[1]-station_lon)
          dist = min(dist,val)
        if dist<0.05:
          continue
        else:
          station_coord.append((station_lat,station_lon))
  return station_coord



#Drawing an animated plot to show county wise location of stations

In [23]:
def graph_county_station_location(big_df,county):
  county_data = big_df[big_df['POOFips']==county][['X','Y','DiscoveryAcres','StartYear','StartMonth','ALAND']].sort_values(by = 'StartYear')
  fig = px.scatter(county_data, x ='Y',y = 'X',
                  color= 'DiscoveryAcres', color_continuous_scale='Reds',
                  animation_frame = 'StartYear')
  station_coord = opt_station_local(county_data)
  points = pd.DataFrame(station_coord,columns = ['x','y'])
  for frame in fig.frames:
      frame.data[0].update({'x': list(frame.data[0].x) + list(points.x),
                            'y': list(frame.data[0].y) + list(points.y)})
      frame.data[0]['mode'] = '+'.join(['markers' for i in range(len(frame.data[0]['x']))])

      sym_list = list(['circle']*(len(frame.data[0]['x'])-len(station_coord) ))
      for i in range(len(station_coord)):
          sym_list.append('star')
      frame.data[0]['marker']['symbol'] = sym_list

  fig.update_layout(yaxis_range=[min(min(county_data['X'].dropna()), min(points['y']))-0.2,max(max(county_data['X'].dropna()), max(points['y']))+0.2],
                    xaxis_range = [min(min(county_data['Y'].dropna()), min(points['x']))-0.2, max(max(county_data['Y'].dropna()), max(points['x'])) +0.2])
  fig.update_traces(marker=dict(size=20,
                                line=dict(width=2,
                                          color='DarkSlateGrey'),
                                reversescale = True),
                    selector=dict(mode='markers'))
  fig.show()
  return station_coord

In [24]:
#Example 
graph_county_station_location(big_df,'04011')

#Now we calculate the area per person

In [103]:
big_df['L_Area_by_pop_2019'] = big_df['ALAND']/big_df['POP_2019']
big_df['L_Area_by_pop_2020'] = big_df['ALAND']/big_df['POP_2020']
big_df['L_Area_by_pop_2021'] = big_df['ALAND']/big_df['POP_2021']
big_df['L_Area_by_pop_2022'] = big_df['ALAND']/big_df['POP_2022']

##Assigning Scores

In [104]:
def normalize(big_df,column):
  return (big_df[column]-min(big_df[column]))/(max(big_df[column])-min(big_df[column]))

big_df['Score'] = 10 - normalize(big_df,'DiscoveryAcres') - normalize(big_df.fillna(0.1),'Rainfall') + normalize(big_df.fillna(50),'Max_Temp') - 0.01*(big_df['StartMonth']-12)*(4+big_df['StartMonth']) + 0.1*big_df['L_Area_by_pop_2022'].fillna(0.001) 
big_df['Score'] = big_df['Score'] + 0.01*(max(big_df['Y'])-big_df['Y']) 
#Normalization of score
big_df['Score'] = big_df['Score']/100

Understanding the wildfire Score:
  1. If there was a big wildfire, then there is a less chance that the same area will get burnt soon. However, for smaller wildfires, or the ones which were contained, there is a greaeter probability of it getting burned again.
  2. Drier weather increases the chances of wildfire. Due to monthly data we look at the rainfall inversely. 
  3. The maximum temperature improves the likelihood of the wildfire
  4. There are more chances of wildfires in the month of April, May, June etc and lesser chances in December and January
  5. The more land available per person implies that there is more nature and possibility of wildfire, both due to humans as well as natural causes.
  6. Finally, going up the latitude decreases the chances of wildfire, due to natural causes to certain extent.
  7. We normalize the score by dividing it by 100 for each incident reported.

#Now that we have the scores for different wildfire incidents, and therefore for the counties as well, let's try to find the stations for some of these counties individually.


In [111]:
#Top 10 most vulnerable counties
top_ten_counties = big_df.groupby('POOFips').sum().sort_values(by = 'Score', ascending = False)['Score'][:10].index


In [ ]:
station_coordinates = []
for county in top_ten_counties:
  station_coordinates.append(graph_county_station_location(big_df,county))